# T1078.004 - Cloud Accounts
Valid accounts in cloud environments may allow adversaries to perform actions to achieve Initial Access, Persistence, Privilege Escalation, or Defense Evasion. Cloud accounts are those created and configured by an organization for use by users, remote support, services, or for administration of resources within a cloud service provider or SaaS application. Cloud Accounts can exist solely in the cloud or be hybrid joined between on-premises systems and the cloud through federation with other identity sources such as Windows Active Directory. (Citation: AWS Identity Federation)(Citation: Google Federating GC)(Citation: Microsoft Deploying AD Federation)

Service or user accounts may be targeted by adversaries through [Brute Force](https://attack.mitre.org/techniques/T1110), [Phishing](https://attack.mitre.org/techniques/T1566), or various other means to gain access to the environment. Federated accounts may be a pathway for the adversary to affect both on-premises systems and cloud environments.

An adversary may create long lasting [Additional Cloud Credentials](https://attack.mitre.org/techniques/T1098/001) on a compromised cloud account to maintain persistence in the environment. Such credentials may also be used to bypass security controls such as multi-factor authentication. 

Cloud accounts may also be able to assume [Temporary Elevated Cloud Access](https://attack.mitre.org/techniques/T1548/005) or other privileges through various means within the environment. Misconfigurations in role assignments or role assumption policies may allow an adversary to use these mechanisms to leverage permissions outside the intended scope of the account. Such over privileged accounts may be used to harvest sensitive data from online storage accounts and databases through [Cloud API](https://attack.mitre.org/techniques/T1059/009) or other methods. 


## Atomic Tests

### Atomic Test #1 - Creating GCP Service Account and Service Account Key
GCP Service Accounts can be used to gain intial access as well as maintain persistence inside Google Cloud.

**Supported Platforms:** google-workspace, iaas:gcp

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Requires gcloud

##### Check Prereq Commands:
```sh
if [ -x "$(command -v gcloud)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
echo "Please Install Google Cloud SDK before running this atomic test : https://cloud.google.com/sdk/docs/install"

```
##### Description: Check if user is logged in 

##### Check Prereq Commands:
```sh
gcloud config get-value account

```
##### Get Prereq Commands:
```sh
gcloud auth login --no-launch-browser

```

In [ ]:
Invoke-AtomicTest T1078.004 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
gcloud config set project art-project-1
gcloud iam service-accounts create gcp-art-service-account-1
gcloud iam service-accounts keys create gcp-art-service-account-1.json --iam-account=gcp-art-service-account-1@art-project-1.iam.gserviceaccount.com
```

In [ ]:
Invoke-AtomicTest T1078.004 -TestNumbers 1

#### Cleanup: 
```sh
gcloud iam service-accounts delete gcp-art-service-account-1@art-project-1.iam.gserviceaccount.com --quiet
```

In [ ]:
Invoke-AtomicTest T1078.004 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Azure Persistence Automation Runbook Created or Modified
Identifies when an Azure Automation runbook is created or modified. An adversary may create or modify an Azure
Automation runbook to execute malicious code and maintain persistence in their target's environment.

**Supported Platforms:** iaas:azure

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Check if terraform is installed.

##### Check Prereq Commands:
```powershell
terraform version

```
##### Get Prereq Commands:
```powershell
echo Please install terraform.

```
##### Description: Install-Module -Name Az

##### Check Prereq Commands:
```powershell
try {if (Get-InstalledModule -Name Az -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name Az -Scope CurrentUser -Force

```
##### Description: Check if the user is logged into Azure.

##### Check Prereq Commands:
```powershell
az account show

```
##### Get Prereq Commands:
```powershell
echo Configure your Azure account using: az login. 

```
##### Description: Create dependency resources using terraform

##### Check Prereq Commands:
```powershell
try {if (Test-Path "$PathToAtomicsFolder/T1078.004/src/T1078.004-2/terraform.tfstate" ){ exit 0 } else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```powershell
cd "$PathToAtomicsFolder/T1078.004/src/T1078.004-2/"
terraform init
terraform apply -auto-approve

```

In [ ]:
Invoke-AtomicTest T1078.004 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$secure_pwd = "None" | ConvertTo-SecureString -AsPlainText -Force
$creds = New-Object System.Management.Automation.PSCredential -ArgumentList "None", $secure_pwd
Connect-AzAccount -Credential $creds
New-AzAutomationRunbook -Name None -Type PowerShell -ResourceGroupName None -Description 'my-test-runbook' -AutomationAccountName None
```

In [ ]:
Invoke-AtomicTest T1078.004 -TestNumbers 2

#### Cleanup: 
```powershell
Remove-AzAutomationRunbook -AutomationAccountName None -Name None -ResourceGroupName None -Force
cd "$PathToAtomicsFolder/T1078.004/src/T1078.004-2/"
terraform destroy -auto-approve
```

In [ ]:
Invoke-AtomicTest T1078.004 -TestNumbers 2 -Cleanup

### Atomic Test #3 - GCP - Create Custom IAM Role
This atomic will create a new IAM role. The default role permissions are: *IAM Service Account Get*. The idea for this Atomic came from a Rule published by the Elastic team.

Identifies an Identity and Access Management (IAM) custom role creation in Google Cloud Platform (GCP). 
Custom roles are user-defined, and allow for the bundling of one or more supported permissions to meet specific needs. 
Custom roles will not be updated automatically and could lead to privilege creep if not carefully scrutinized.

This atomic will create a new IAM role. The default role permissions are: *IAM Service Account Get*

Reference: https://github.com/elastic/detection-rules/blob/main/rules/integrations/gcp/initial_access_gcp_iam_custom_role_creation.toml

**Supported Platforms:** iaas:gcp

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Requires gcloud

##### Check Prereq Commands:
```sh
if [ -x "$(command -v gcloud)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
echo "Please Install Google Cloud SDK before running this atomic test : https://cloud.google.com/sdk/docs/install"

```
##### Description: Check if user is logged in 

##### Check Prereq Commands:
```sh
gcloud config get-value account

```
##### Get Prereq Commands:
```sh
gcloud auth login --no-launch-browser

```

In [ ]:
Invoke-AtomicTest T1078.004 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
gcloud config set project atomic-test-1
gcloud iam roles create AtomicRedTeamRole --description="Atomic Red Team Custom IAM Role" --permissions=iam.serviceAccounts.get --project=atomic-test-1
```

In [ ]:
Invoke-AtomicTest T1078.004 -TestNumbers 3

#### Cleanup: 
```sh
gcloud iam roles delete AtomicRedTeamRole --project=atomic-test-1
```

In [ ]:
Invoke-AtomicTest T1078.004 -TestNumbers 3 -Cleanup

## Detection
Monitor the activity of cloud accounts to detect abnormal or malicious behavior, such as accessing information outside of the normal function of the account or account usage at atypical hours.